## Repeated Cross Validation for broad category model

The notebook manages the main pipeline for models training/testing cross validation.
* Reads the annotated dataset
* Select only the largets broad categories that have enough data for training
* Transforms the data into format for training spaCy models
* It split the training data into chunks according to the cross validations folds settings
* Generates the jsonl input files
* It executes the pipeline by training and testing accross all chunks of data

The main pipeline is contained in the folder 'textcat_broad_categories' and it's set up to accept the input files:
* assays_train.jsonl
* assays_test.jsonl

After execution of input data it generates the following outputs:
* model-best
* model-last
* metrics.json

While executing CV, each of the chunks data inputs is copied to the pipeline folder. The pipeline is executed with those files and outputs generated are copied back out to the specified path.

For live testing of a specific model, the output files of such model can be moved to the output folder and execute the pipeline commands.

### Importing modules

In [1]:
import pandas as pd
from pathlib import Path
import os
import json
import spacy
from sklearn.model_selection import RepeatedKFold
import shutil
import subprocess

### Setup

In [2]:
# Set path to python in the environment to use for model training
env_path = "/hps/software/users/chembl/ines/assays_description/bin/python"

In [3]:
#Settings for display (if needed)
pd.set_option('display.max_colwidth', None)  # Set to None to display the full column width
pd.set_option('display.max_rows', None)      # Set to None to display al

### Model training: Binding assays and functional assays

##### Main annotated dataset

In [4]:
dataset = pd.read_csv('data/2_broad_category_training_data.csv')

In [5]:
dataset.head()

,assay_id,assay_type,description,label,bao_preferred_term,bao_id
0,868,B,"Inhibition of [3H]8-hydroxy-2-dipropylamino-1,2,3,4-tetrahydronaphthalene binding to 5-hydroxytryptamine 1A receptor in hippocampus region of rat brain; Residual radioligand binding higher than 50%","Radioligand competition, displacement, binding",radioligand binding assay,BAO_0002776
1,2027,B,Displacement of [3H]-5-HT from human 5-hydroxytryptamine 1D receptor beta,"Radioligand competition, displacement, binding",radioligand binding assay,BAO_0002776
2,2430,B,Inhibition constant for in vitro inhibition of [3H]ketanserin binding to rat frontal cortex membranes 5-hydroxytryptamine 2A receptor,"Radioligand competition, displacement, binding",radioligand binding assay,BAO_0002776
3,3306,B,Compound was evaluated for the binding affinity against human cloned 5-hydroxytryptamine 4 receptor in HeLa cells using [3H]-LSD as the radioligand,"Radioligand competition, displacement, binding",radioligand binding assay,BAO_0002776
4,3703,B,In vitro binding affinity by radioligand binding assay using cell line expressing human 5-hydroxytryptamine 7 receptor; ND means not determined,"Radioligand competition, displacement, binding",radioligand binding assay,BAO_0002776


In [6]:
### Check annotated broad categories
dataset['label'].value_counts()

label
Nucleic acid binding                              176
Protein activity                                  171
Binding affinity, displacement, competition       134
Radioligand competition, displacement, binding    118
Cell phenotype                                    113
Antimicrobial activity                             88
in vivo method                                     63
Name: count, dtype: int64

In [7]:
len(dataset)

863

In [8]:
# Can only train the large categories
chosen_categories = [
    'Nucleic acid binding'
    , 'Protein activity'
    , 'Binding affinity, displacement, competition'
    , 'Radioligand competition, displacement, binding'
    , 'Cell phenotype'
    , 'Antimicrobial activity'
    , 'in vivo method'
]

In [9]:
processed_df = dataset.loc[dataset['label'].isin(chosen_categories)]

In [10]:
len(dataset), len(processed_df)

(863, 863)

In [11]:
dataset_counts = pd.DataFrame(data=processed_df['label'].value_counts().reset_index())
dataset_counts.rename(columns={'count': 'assay_count'}, inplace=True)
dataset_counts

,label,assay_count
0,Nucleic acid binding,176
1,Protein activity,171
2,"Binding affinity, displacement, competition",134
3,"Radioligand competition, displacement, binding",118
4,Cell phenotype,113
5,Antimicrobial activity,88
6,in vivo method,63


In [12]:
labels_ids = processed_df[['label', 'bao_preferred_term', 'bao_id']].drop_duplicates()
labels_ids

,label,bao_preferred_term,bao_id
0,"Radioligand competition, displacement, binding",radioligand binding assay,BAO_0002776
6,Protein activity,functional target-based,BAO_0013016
12,"Binding affinity, displacement, competition",binding assay,BAO_0002989
16,in vivo method,in vivo assay method,BAO_0040021
22,Cell phenotype,cell phenotype,BAO_0002542
28,Nucleic acid binding,NaN,No suitable ID at present
41,Antimicrobial activity,NaN,No suitable ID at present


In [13]:
dataset_counts_merged = dataset_counts.merge(labels_ids, on = 'label')
dataset_counts_merged

,label,assay_count,bao_preferred_term,bao_id
0,Nucleic acid binding,176,NaN,No suitable ID at present
1,Protein activity,171,functional target-based,BAO_0013016
2,"Binding affinity, displacement, competition",134,binding assay,BAO_0002989
3,"Radioligand competition, displacement, binding",118,radioligand binding assay,BAO_0002776
4,Cell phenotype,113,cell phenotype,BAO_0002542
5,Antimicrobial activity,88,NaN,No suitable ID at present
6,in vivo method,63,in vivo assay method,BAO_0040021


##### Process input data to the format for training in spaCy

In [14]:
value_dict = {
    'Radioligand competition, displacement, binding': {'Radioligand binding (BAO_0002776)': 1.0, 'Binding (BAO_0002989)': 0.0, 'Protein activity (BAO_0013016)': 0.0, 'in vivo method (BAO_0040021)': 0.0, 'Cell phenotype (BAO_0002542)': 0.0, 'Nucleic acid binding': 0.0, 'Antimicrobial activity': 0.0}
    , 'Binding affinity, displacement, competition': {'Radioligand binding (BAO_0002776)': 0.0, 'Binding (BAO_0002989)': 1.0, 'Protein activity (BAO_0013016)': 0.0, 'in vivo method (BAO_0040021)': 0.0, 'Cell phenotype (BAO_0002542)': 0.0, 'Nucleic acid binding': 0.0, 'Antimicrobial activity': 0.0}
    , 'Protein activity': {'Radioligand binding (BAO_0002776)': 0.0, 'Binding (BAO_0002989)': 0.0, 'Protein activity (BAO_0013016)': 1.0, 'in vivo method (BAO_0040021)': 0.0, 'Cell phenotype (BAO_0002542)': 0.0, 'Nucleic acid binding': 0.0, 'Antimicrobial activity': 0.0}
    , 'in vivo method': {'Radioligand binding (BAO_0002776)': 0.0, 'Binding (BAO_0002989)': 0.0, 'Protein activity (BAO_0013016)': 0.0, 'in vivo method (BAO_0040021)': 1.0, 'Cell phenotype (BAO_0002542)': 0.0, 'Nucleic acid binding': 0.0, 'Antimicrobial activity': 0.0}
    , 'Cell phenotype': {'Radioligand binding (BAO_0002776)': 0.0, 'Binding (BAO_0002989)': 0.0, 'Protein activity (BAO_0013016)': 0.0, 'in vivo method (BAO_0040021)': 0.0, 'Cell phenotype (BAO_0002542)': 1.0, 'Nucleic acid binding': 0.0, 'Antimicrobial activity': 0.0}
    , 'Nucleic acid binding': {'Radioligand binding (BAO_0002776)': 0.0, 'Binding (BAO_0002989)': 0.0, 'Protein activity (BAO_0013016)': 0.0, 'in vivo method (BAO_0040021)': 0.0, 'Cell phenotype (BAO_0002542)': 0.0, 'Nucleic acid binding': 1.0, 'Antimicrobial activity': 0.0}
    , 'Antimicrobial activity': {'Radioligand binding (BAO_0002776)': 0.0, 'Binding (BAO_0002989)': 0.0, 'Protein activity (BAO_0013016)': 0.0, 'in vivo method (BAO_0040021)': 0.0, 'Cell phenotype (BAO_0002542)': 0.0, 'Nucleic acid binding': 0.0, 'Antimicrobial activity': 1.0}

}

In [15]:
processed_df['cats'] = (
    processed_df['label']
    .apply(lambda x: value_dict[x])
)

/tmp/ipykernel_759787/479963469.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_df['cats'] = (


In [16]:
processed_df.rename(columns={'description': 'text'}, inplace=True)

/tmp/ipykernel_759787/3262683888.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_df.rename(columns={'description': 'text'}, inplace=True)


In [17]:
processed_df = processed_df[['text', 'cats']]

In [18]:
processed_df

,text,cats
0,"Inhibition of [3H]8-hydroxy-2-dipropylamino-1,2,3,4-tetrahydronaphthalene binding to 5-hydroxytryptamine 1A receptor in hippocampus region of rat brain; Residual radioligand binding higher than 50%","{'Radioligand binding (BAO_0002776)': 1.0, 'Binding (BAO_0002989)': 0.0, 'Protein activity (BAO_0013016)': 0.0, 'in vivo method (BAO_0040021)': 0.0, 'Cell phenotype (BAO_0002542)': 0.0, 'Nucleic acid binding': 0.0, 'Antimicrobial activity': 0.0}"
1,Displacement of [3H]-5-HT from human 5-hydroxytryptamine 1D receptor beta,"{'Radioligand binding (BAO_0002776)': 1.0, 'Binding (BAO_0002989)': 0.0, 'Protein activity (BAO_0013016)': 0.0, 'in vivo method (BAO_0040021)': 0.0, 'Cell phenotype (BAO_0002542)': 0.0, 'Nucleic acid binding': 0.0, 'Antimicrobial activity': 0.0}"
2,Inhibition constant for in vitro inhibition of [3H]ketanserin binding to rat frontal cortex membranes 5-hydroxytryptamine 2A receptor,"{'Radioligand binding (BAO_0002776)': 1.0, 'Binding (BAO_0002989)': 0.0, 'Protein activity (BAO_0013016)': 0.0, 'in vivo method (BAO_0040021)': 0.0, 'Cell phenotype (BAO_0002542)': 0.0, 'Nucleic acid binding': 0.0, 'Antimicrobial activity': 0.0}"
3,Compound was evaluated for the binding affinity against human cloned 5-hydroxytryptamine 4 receptor in HeLa cells using [3H]-LSD as the radioligand,"{'Radioligand binding (BAO_0002776)': 1.0, 'Binding (BAO_0002989)': 0.0, 'Protein activity (BAO_0013016)': 0.0, 'in vivo method (BAO_0040021)': 0.0, 'Cell phenotype (BAO_0002542)': 0.0, 'Nucleic acid binding': 0.0, 'Antimicrobial activity': 0.0}"
4,In vitro binding affinity by radioligand binding assay using cell line expressing human 5-hydroxytryptamine 7 receptor; ND means not determined,"{'Radioligand binding (BAO_0002776)': 1.0, 'Binding (BAO_0002989)': 0.0, 'Protein activity (BAO_0013016)': 0.0, 'in vivo method (BAO_0040021)': 0.0, 'Cell phenotype (BAO_0002542)': 0.0, 'Nucleic acid binding': 0.0, 'Antimicrobial activity': 0.0}"
6,Antagonism of NECA-induced stimulation of adenylate cyclase activity in human platelet membranes at A2-adenosine receptor; value ranges from 21-30,"{'Radioligand binding (BAO_0002776)': 0.0, 'Binding (BAO_0002989)': 0.0, 'Protein activity (BAO_0013016)': 1.0, 'in vivo method (BAO_0040021)': 0.0, 'Cell phenotype (BAO_0002542)': 0.0, 'Nucleic acid binding': 0.0, 'Antimicrobial activity': 0.0}"
7,Inhibitory activity was calculated for the model Acetylcholinesterase (Expt-1),"{'Radioligand binding (BAO_0002776)': 0.0, 'Binding (BAO_0002989)': 0.0, 'Protein activity (BAO_0013016)': 1.0, 'in vivo method (BAO_0040021)': 0.0, 'Cell phenotype (BAO_0002542)': 0.0, 'Nucleic acid binding': 0.0, 'Antimicrobial activity': 0.0}"
8,Percent remaining of radioligand [3H]DPCPX binding to human adenosine A1 receptor at 10 uM,"{'Radioligand binding (BAO_0002776)': 1.0, 'Binding (BAO_0002989)': 0.0, 'Protein activity (BAO_0013016)': 0.0, 'in vivo method (BAO_0040021)': 0.0, 'Cell phenotype (BAO_0002542)': 0.0, 'Nucleic acid binding': 0.0, 'Antimicrobial activity': 0.0}"
9,Concentration required for 50% inhibition of [3H]NECA binding on rat brain adenosine A2 receptor,"{'Radioligand binding (BAO_0002776)': 1.0, 'Binding (BAO_0002989)': 0.0, 'Protein activity (BAO_0013016)': 0.0, 'in vivo method (BAO_0040021)': 0.0, 'Cell phenotype (BAO_0002542)': 0.0, 'Nucleic acid binding': 0.0, 'Antimicrobial activity': 0.0}"
10,In vitro concentration required for 50% inhibition against Adenosine Kinase (AK) in the presence of intact cells,"{'Radioligand binding (BAO_0002776)': 0.0, 'Binding (BAO_0002989)': 0.0, 'Protein activity (BAO_0013016)': 1.0, 'in vivo method (BAO_0040021)': 0.0, 'Cell phenotype (BAO_0002542)': 0.0, 'Nucleic acid binding': 0.0, 'Antimicrobial activity': 0.0}"


##### Setting up the Cross-Validation

In [19]:
mpath = "Model_cv"
dirpath = Path(mpath)
if dirpath.exists():
    shutil.rmtree(dirpath)
os.makedirs(mpath)

#Method to generate the cross validation chunks
def save_CV_folds_json(mpath, dataset):
    
    rkf = RepeatedKFold(n_splits=5, n_repeats=5, random_state=573927565)
    for fold, (train_index, test_index) in enumerate(rkf.split(dataset)):
        train_df = dataset.iloc[train_index]
        test_df = dataset.iloc[test_index]

        path = os.path.join(mpath,'chunk{}'.format(fold))
        os.makedirs(path, exist_ok=True)
        
        # Write to JSONL files
        with open(os.path.join(path,'assays_train.jsonl'), 'w') as f:
            f.write(train_df.to_json(orient='records', lines=True))
        with open(os.path.join(path,'assays_test.jsonl'), 'w') as f:
            f.write(test_df.to_json(orient='records', lines=True))

In [20]:
save_CV_folds_json(mpath, processed_df)

##### Executing Cross-Validation: Move inputs to pipeline, run pipeline, take outputs

In [21]:
for f in Path(mpath).iterdir():
    
    chunk = f.name
    print(chunk)
    train = os.path.join('../', f,'assays_train.jsonl')
    test = os.path.join('../', f,'assays_test.jsonl')

    # Chdir to the pipeline template folder
    os.chdir('textcat_broad_categories')
    
    os.mkdir('assets')
    os.mkdir('training')

    #Copy the current input files to the pipeline path
    shutil.copy(train, os.path.join('assets/'))
    shutil.copy(test, os.path.join('assets/'))

    #Run the pipeline
    command = f'{env_path} -m weasel run all'
    subprocess.run(command, shell=True, capture_output=False, text=True)
    
    #Move outputs to the chunk folder
    opath = os.path.join('../', f, 'training')
    shutil.copytree('training', opath, dirs_exist_ok=True)

    # Remove directories to start clean
    shutil.rmtree('assets')
    shutil.rmtree('training')
    shutil.rmtree('corpus')
    os.remove('project.lock')

    os.chdir('../')


chunk0


ℹ Saving to output directory: training
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'textcat_multilabel']
ℹ Initial learn rate: 0.0
E    #       LOSS TOK2VEC  LOSS TEXTC...  CATS_SCORE  SCORE 
---  ------  ------------  -------------  ----------  ------


  0       0          0.63           0.36       55.79    0.56


  1     200         91.61          60.29       61.17    0.61


  3     400         45.76          28.94       79.86    0.80


...

210    4600          6.38           0.13       98.02    0.98


224    4800          3.76           0.09       98.10    0.98


237    5000          4.36           0.07       98.06    0.98


250    5200          2.33           0.05       98.03    0.98
✔ Saved pipeline to output directory
training/model-last


ℹ Using CPU

================================== Results ==================================

TOK                   100.00
TEXTCAT (macro AUC)   98.29 
SPEED                 27661 


=========================== Textcat F (per label) ===========================

                                      P        R       F
Radioligand binding (BAO_0002776)   100.00    96.55   98.25
Binding (BAO_0002989)             81.82   100.00   90.00
Protein activity (BAO_0013016)    91.30    65.62   76.36
in vivo method (BAO_0040021)     100.00    85.71   92.31
Cell phenotype (BAO_0002542)      85.00    85.00   85.00
Nucleic acid binding             100.00    89.47   94.44
Antimicrobial activity            94.44    80.95   87.18


======================== Textcat ROC AUC (per label) ========================

                                 ROC AUC
Radioligand binding (BAO_0002776)      0.98
Binding (BAO_0002989)               0.99
Protein activity (BAO_0013016)      0.95
in vivo method (BAO_0040021)     

##### Cross-Validation metrics

In [22]:
metrics_all = []

for f in Path(mpath).iterdir():
    if not f.name.endswith('.csv'):
        chunk = f.name
        metricspath = os.path.join(f,'training/metrics.json')
        # Open JSON File
        with open(metricspath, 'r') as file:
            data = json.load(file)['cats_f_per_type']
            df = pd.DataFrame.from_dict(data, orient='index')
            df = df.map(lambda x: format(x, ".2f"))
            df['fold'] = chunk
            df.reset_index(inplace=True)
            df.rename(columns={'index': 'category'}, inplace=True)
            metrics_all.append(df)

In [23]:
pd.concat(metrics_all).sort_values(by='category').to_csv('./results/models_metrics.csv', index=False)